In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://calendar.google.com/calendar/u/0/embed?src=7a04205ae91b85e82ebc74daddbf2933c6b6723b81abb966f0e69c66a996c43b@group.calendar.google.com&ctz=America/Vancouver&pli=1).**

## Imports

In [2]:
import os
from hashlib import sha1

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

## Submission instructions
<hr>
rubric={points:2}

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **You may work on this assignment in a group (group size <= 4) and submit your assignment as a group.** 
- Below are some instructions on working as a group.  
    - The maximum group size is 4. 
    - You can choose your own group members. 
    - Use group work as an opportunity to collaborate and learn new things from each other. 
    - Be respectful to each other and make sure you understand all the concepts in the assignment well. 
    - It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. [Here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members) are some instructions on adding group members in Gradescope.  
- Upload the .ipynb file to Gradescope.
- **If the .ipynb file is too big or doesn't render on Gradescope for some reason, also upload a pdf or html in addition to the .ipynb.** 
- Make sure that your plots/output are rendered properly in Gradescope.

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points:2}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words in `glove_wiki_vectors` embeddings. 

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:
gwv = glove_wiki_vectors

In [7]:
words = ["hat", "banana", "dog", "car"]

In [8]:
for word in words:
    print(gwv.most_similar(word))

[('hats', 0.7115386724472046), ('shirt', 0.6908950209617615), ('wears', 0.6533207297325134), ('outfit', 0.6489425301551819), ('trick', 0.642845630645752), ('boots', 0.6397153735160828), ('wore', 0.6366178393363953), ('jacket', 0.6342592835426331), ('wearing', 0.6320689916610718), ('scarf', 0.6214569211006165)]
[('coconut', 0.7097253203392029), ('mango', 0.7054824233055115), ('bananas', 0.6887733936309814), ('potato', 0.6629636287689209), ('pineapple', 0.6534532308578491), ('fruit', 0.6519854664802551), ('peanut', 0.6420575976371765), ('pecan', 0.6349172592163086), ('cashew', 0.6294420957565308), ('papaya', 0.6246591210365295)]
[('cat', 0.8798075914382935), ('dogs', 0.8344309329986572), ('pet', 0.7449564337730408), ('puppy', 0.723637580871582), ('horse', 0.7109652757644653), ('animal', 0.6817062497138977), ('pig', 0.6554172039031982), ('boy', 0.6545308232307434), ('cats', 0.6471933126449585), ('rabbit', 0.6468630433082581)]
[('vehicle', 0.8630837798118591), ('truck', 0.8597878813743591)

In [9]:
similar_words = ["hats", "coconut", "cat", "vehicle"]

<!-- END QUESTION -->

<br><br>

### 1.2 Word similarity using pre-trained embeddings
rubric={autograde:2}

**Your tasks:**
1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [10]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [11]:
coast_shore_similarity = gwv.similarity(word_pairs[0][0], word_pairs[0][1])
clothes_closet_similarity = gwv.similarity(word_pairs[1][0], word_pairs[1][1])
old_new_similarity = gwv.similarity(word_pairs[2][0], word_pairs[2][1])
smart_intelligent_similarity = gwv.similarity(word_pairs[3][0], word_pairs[3][1])
dog_cat_similarity = gwv.similarity(word_pairs[4][0], word_pairs[4][1])
tree_lawyer_similarity = gwv.similarity(word_pairs[5][0], word_pairs[5][1])

In [12]:
grader.check("q1.2")

q1.2 results: All test cases passed!

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Stereotypes and biases in embeddings
rubric={points:6}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings or not by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore what kind of stereotypes and biases are encoded in these embeddings. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)   
2. Discuss your observations.

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

An example of using analogy to explore biases and stereotypes.

In [13]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

In [14]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


An example of using similarity between words to explore biases and stereotypes.  

In [15]:
glove_wiki_vectors.similarity("white", "rich")

0.44723597

In [16]:
glove_wiki_vectors.similarity("black", "rich")

0.517452

<div class="alert alert-warning">

Solution_1.3
    
</div>

_Points:_ 6

In [17]:
gwv.similarity("woman", "engineer"), gwv.similarity("man", "engineer")

(0.3340311, 0.42998508)

In [18]:
gwv.similarity("woman", "nurse"), gwv.similarity("man", "nurse")

(0.61394423, 0.4562388)

In [19]:
gwv.similarity("woman", "construction"), gwv.similarity("man", "construction")

(0.23188335, 0.29262656)

In [20]:
analogy("woman", "nurse", "man")

woman : nurse :: man : ?


,Analogy word,Score
0,doctor,0.720885
1,physician,0.603826
2,officer,0.598570
3,hospital,0.597952
4,surgeon,0.593788
5,trained,0.586958
6,psychiatrist,0.586677
7,technician,0.585350
8,sergeant,0.584288
9,nurses,0.583345


2. I explored some biases and stereotypes in occupational differences between men and women.  There is a noticeable skew where male-dominated fields such as engineers, doctors, and construction workers have higher similarity men than women, and female dominated fields such as nursing have higher similarity with women.  This is not too surprising when considering that the words likely tend to occur together more frequently in reflection of the distribution in the real world.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Discussion
rubric={points:4}

**Your tasks:**
1. Based on your exploration above, comment on the overall quality of these pre-trained embeddings. 
2. In the lecture, we saw that our pre-trained word embedding model output an analogy that reinforced a gender stereotype. Give an example of how using such a model could cause harm in the real world.

<div class="alert alert-warning">

Solution_1.4
    
</div>

_Points:_ 4

1. From the exploration above, the pre-trained embeddings seem to be great in quality.  The results of functions like most_similar, similarity, and analogy are performing to expectations and make sense.
2. For example, if there were a company that was hiring candidates with the use of AI tooling, such biases can affect recommendations and preference could be given to males in male-biased professions.

<!-- END QUESTION -->

<br><br>

### 1.5 Classification with pre-trained embeddings 
rubric={autograde:8}

In lecture 18, we saw that you can conveniently get word vectors with `spaCy` with `en_core_web_md` model. In this exercise, you'll use word embeddings in multi-class text classification task. We will use [HappyDB](https://www.kaggle.com/ritresearch/happydb) corpus which contains about 100,000 happy moments classified into 7 categories: *affection, exercise, bonding, nature, leisure, achievement, enjoy_the_moment*. The data was crowd-sourced via [Amazon Mechanical Turk](https://www.mturk.com/). The ground truth label is not available for all examples, and in this lab, we'll only use the examples where ground truth is available (~15,000 examples). 

- Download the data from [here](https://www.kaggle.com/ritresearch/happydb).
- Unzip the file and copy it under the data/ directory in this homework directory.

The code below reads the data CSV (assuming that it's present in the current directory as *cleaned_hm.csv*),  cleans it up a bit, and splits it into train and test splits. 

**Your tasks:**

1. Train logistic regression with bag-of-words features (`CountVectorizer`, be sure to filter out stop words), calculate the train and test accuracy and show a [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set.
2. Train logistic regression with average embedding representation extracted using spaCy, calculate the train and test accuracy and show [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set. (You can refer to lecture 18 notes for this.)  

In [21]:
df = pd.read_csv("data/cleaned_hm.csv", index_col=0)
sample_df = df.dropna()
sample_df.head()

,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
hmid,,,,,,,,
27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
27678,45,24h,I meditated last night.,I meditated last night.,True,1,leisure,leisure
27697,498,24h,My grandmother start to walk from the bed afte...,My grandmother start to walk from the bed afte...,True,1,affection,affection
27705,5732,24h,I picked my daughter up from the airport and w...,I picked my daughter up from the airport and w...,True,1,bonding,affection
27715,2272,24h,when i received flowers from my best friend,when i received flowers from my best friend,True,1,bonding,bonding


In [22]:
sample_df = sample_df.rename(
    columns={"cleaned_hm": "moment", "ground_truth_category": "target"}
)

In [23]:
train_df, test_df = train_test_split(sample_df, test_size=0.3, random_state=123)
X_train, y_train = train_df["moment"], train_df["target"]
X_test, y_test = test_df["moment"], test_df["target"]

You need `spacy` to run the code below. If it's not in your course conda environment, you need to install it. 

> conda install -c conda-forge spacy

You also need to download the following language model. 

> python -m spacy download en_core_web_md

In [24]:
import spacy
import en_core_web_md

# nlp = spacy.load("en_core_web_md")
nlp = en_core_web_md.load()

<div class="alert alert-warning">

Solution_1.5
    
</div>

_Points:_ 8

_Type your answer here, replacing this text._

In [25]:
from sklearn.metrics import classification_report

In [26]:
vec = CountVectorizer(stop_words="english")
X_train_trans = vec.fit_transform(X_train, y_train)
X_test_trans = vec.transform(X_test)

In [27]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_trans, y_train)

LogisticRegression(max_iter=1000)

In [28]:
bow_train_accuracy = lr.score(X_train_trans, y_train)
bow_test_accuracy = lr.score(X_test_trans, y_test)
bow_report = classification_report(y_test, lr.predict(X_test_trans))

In [29]:
X_train_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_train)])
X_test_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_test)])

In [30]:
lr = LogisticRegression(max_iter=2000)
lr.fit(X_train_embeddings, y_train)

C:\Users\ryanm\.conda\envs\cpsc330\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=2000)

In [31]:
emb_train_accuracy = lr.score(X_train_embeddings, y_train)
emb_test_accuracy = lr.score(X_test_embeddings, y_test)
emb_report = classification_report(y_test, lr.predict(X_test_embeddings))
emb_train_accuracy,emb_test_accuracy,emb_report

# from Piazza post @421
from math import floor
emb_train_accuracy = floor(emb_train_accuracy*1000)/1000

In [32]:
grader.check("q1.5")

q1.5 results: All test cases passed!

<br><br>

<!-- BEGIN QUESTION -->

### 1.6 Discussion
rubric={points:6}

**Your tasks:**
1. Briefly explain the difference between using `CountVectorizer` vs. average-embedding approach for text classification.  
2. Which representation among these two would be more interpretable? Why?   
3. Are we using any transfer learning here? If yes, are you observing any benefits of transfer learning? Briefly discuss. 

<div class="alert alert-warning">

Solution_1.6
    
</div>

_Points:_ 6

In [33]:
bow_report

'                  precision    recall  f1-score   support\n\n     achievement       0.79      0.87      0.83      1302\n       affection       0.90      0.91      0.91      1423\n         bonding       0.91      0.85      0.88       492\nenjoy_the_moment       0.60      0.54      0.57       469\n        exercise       0.91      0.57      0.70        74\n         leisure       0.73      0.70      0.71       407\n          nature       0.73      0.46      0.57        71\n\n        accuracy                           0.82      4238\n       macro avg       0.80      0.70      0.74      4238\n    weighted avg       0.82      0.82      0.81      4238\n'

In [34]:
emb_report

'                  precision    recall  f1-score   support\n\n     achievement       0.81      0.83      0.82      1302\n       affection       0.86      0.91      0.88      1423\n         bonding       0.83      0.76      0.79       492\nenjoy_the_moment       0.56      0.51      0.53       469\n        exercise       0.67      0.73      0.70        74\n         leisure       0.71      0.65      0.68       407\n          nature       0.67      0.72      0.69        71\n\n        accuracy                           0.79      4238\n       macro avg       0.73      0.73      0.73      4238\n    weighted avg       0.79      0.79      0.79      4238\n'

1. CountVectorizer represents text by counting the occurrence of words, while average-embedding uses pre-trained word embeddings to capture the semantic meaning of words. CountVectorizer ignores word order and context, while average-embedding considers the similarity between words based on their embeddings.
2. CountVectorizer seems to be more interpretable because it is very simply counting the number of occurrences of a word in any given text, which is easier to understand intuitively.  In contrast, embedding uses vector representations which are harder to intuit.
3. Yes, we used pre-trained word embeddings from spaCy and en_core_web_md in the average-embedding approach.  We observed great benefits, including skipping training time, having a good model trained on a large dataset, and simple convenience of having a pre-trained model.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The overarching goal of topic modeling is understanding high-level themes in a large collection of texts in an unsupervised way. 

In this exercise you will explore topics in a subset of `scikit-learn`'s [20 newsgroups text dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [35]:
from sklearn.datasets import fetch_20newsgroups

In [36]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [37]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points:8}

In this exercise you'll prepare the data for topic modeling using [spaCy](https://spacy.io/). Preprocessing is a crucial step before training an LDA model and it markedly affects topic modeling results. So let's carry out preprocessing. 

**Your tasks:** 

Write code to carry out preprocessing of the "text" column in the dataframe above and store the preprocessed text in a new column called "text_pp" in the dataframe. 


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are more informative and which ones are less informative for the given task. Some common text preprocessing steps for topic modeling are: 
- getting rid of slashes or other weird characters
- sentence segmentation and tokenization      
- getting rid of urls and email addresses
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization        

You might have to go back and forth between the preprocessing and topic modeling and interpretation steps in the next exercises. 

> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://spacy.io/api/annotation/#pos-en) is the list of part-of-speech tags used in spaCy. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you're done with the preprocessing part, you might want to save the preprocessed data so that you don't run the preprocessing part every time you run the notebook. 


<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

_Type your answer here, replacing this text._

In [38]:
text = df["text"]
rows = text.shape[0]
text

0       You know, I was reading 18 U.S.C. 922 and some...
1       \n\n\nIt's not a bad question: I don't have an...
2       \nActuallay I don't, but on the other hand I d...
3       The following problem is really bugging me,\na...
4       \n\n  This is the latest from UPI \n\n     For...
                              ...                        
4558    Hi Everyone ::\n\nI am  looking for  some soft...
4559    Archive-name: x-faq/part3\nLast-modified: 1993...
4560    \nThat's nice, but it doesn't answer the quest...
4561    Hi,\n     I just got myself a Gateway 4DX-33V ...
4562    \n\n[h] \tThe Armenians in Nagarno-Karabagh ar...
Name: text, Length: 4563, dtype: object

In [39]:
# from nltk.tokenize import sent_tokenize
import copy

# sent_tokenized = copy.deepcopy(text)
# for i in range(rows):
#     sent_tokenized[i] = sent_tokenize(text[i])
# sent_tokenized

In [40]:
from nltk.tokenize import word_tokenize

word_tokenized = copy.deepcopy(text)
for i in range(rows):
    word_tokenized[i] = word_tokenize(text[i])
word_tokenized

0       [You, know, ,, I, was, reading, 18, U.S.C, ., ...
1       [It, 's, not, a, bad, question, :, I, do, n't,...
2       [Actuallay, I, do, n't, ,, but, on, the, other...
3       [The, following, problem, is, really, bugging,...
4       [This, is, the, latest, from, UPI, Foreign, Mi...
                              ...                        
4558    [Hi, Everyone, :, :, I, am, looking, for, some...
4559    [Archive-name, :, x-faq/part3, Last-modified, ...
4560    [That, 's, nice, ,, but, it, does, n't, answer...
4561    [Hi, ,, I, just, got, myself, a, Gateway, 4DX-...
4562    [[, h, ], The, Armenians, in, Nagarno-Karabagh...
Name: text, Length: 4563, dtype: object

In [41]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ryanm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
stop_words = list(set(stopwords.words("english")))

import string

punctuation = string.punctuation
stop_words += list(punctuation)
stop_words.extend(['``','`','br','"',"”", "''", "'s", "n't", "like", "thing", "say", "think", "know", "said", "use"])
print(stop_words)

["you'll", 'what', 'my', 'before', 'are', 'once', 'doesn', 'them', 'myself', 'mightn', 'm', 'whom', 'having', 'under', 'was', "didn't", "weren't", 'such', "wasn't", 'about', 'on', 'but', 'am', 'nor', "shouldn't", "don't", 'll', 'isn', 'ourselves', 'with', 'his', 'hadn', 'again', 'while', 'above', 'have', 'after', 'or', 'further', 'haven', 'we', 'more', 'not', 'couldn', 'is', 'why', 'herself', 'so', 'which', 'those', 'needn', 'our', 'during', 'then', 'when', 'should', 'between', 'won', 'that', 'up', 'down', 'ma', 'out', "shan't", 'who', 'now', 'd', 'all', 'from', 'these', 'were', 'does', 'just', 'o', 'don', 'into', 'themselves', "it's", 'same', 'had', 'be', 'here', 'a', 'your', 'has', 'how', "mustn't", 'their', 'of', "hasn't", 'other', 'did', 're', 'wouldn', 'do', 'an', 'hers', "you've", 'wasn', 'yourself', 'me', "wouldn't", 'been', 'didn', "that'll", 'off', 'some', 'than', 'y', "hadn't", "you'd", 'can', 'itself', 'both', 'each', 't', 'over', 'if', 'in', 'to', 'too', 'against', 'they', 

In [43]:
words_stopped = copy.deepcopy(word_tokenized)
for i in range(rows):
    words_stopped[i] = []
    for token in word_tokenized[i]:
        token = token.lower()
        if token not in stop_words and not token.isdigit():
            words_stopped[i].append(token)
words_stopped

0       [reading, u.s.c, something, make, sence, wonde...
1       [bad, question, refs, list, algorithm, either,...
2       [actuallay, hand, support, idea, one, newsgrou...
3       [following, problem, really, bugging, would, a...
4       [latest, upi, foreign, ministry, spokesman, fe...
                              ...                        
4558    [hi, everyone, looking, software, called, shad...
4559    [archive-name, x-faq/part3, last-modified, 199...
4560    [nice, answer, question, difference, feds, man...
4561    [hi, got, gateway, 4dx-33v, trying, configure,...
4562    [h, armenians, nagarno-karabagh, simply, defen...
Name: text, Length: 4563, dtype: object

In [44]:
nltk.download("wordnet")
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ryanm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ryanm\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [45]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

lem = copy.deepcopy(words_stopped)

for i in range(rows):
    lem[i] = []
    for token in words_stopped[i]:
        token = lemmatizer.lemmatize(token)
        lem[i].append(token)
lem

0       [reading, u.s.c, something, make, sence, wonde...
1       [bad, question, ref, list, algorithm, either, ...
2       [actuallay, hand, support, idea, one, newsgrou...
3       [following, problem, really, bugging, would, a...
4       [latest, upi, foreign, ministry, spokesman, fe...
                              ...                        
4558    [hi, everyone, looking, software, called, shad...
4559    [archive-name, x-faq/part3, last-modified, 199...
4560    [nice, answer, question, difference, fed, mand...
4561    [hi, got, gateway, 4dx-33v, trying, configure,...
4562    [h, armenian, nagarno-karabagh, simply, defend...
Name: text, Length: 4563, dtype: object

In [46]:
text_pp = lem
text_pp

0       [reading, u.s.c, something, make, sence, wonde...
1       [bad, question, ref, list, algorithm, either, ...
2       [actuallay, hand, support, idea, one, newsgrou...
3       [following, problem, really, bugging, would, a...
4       [latest, upi, foreign, ministry, spokesman, fe...
                              ...                        
4558    [hi, everyone, looking, software, called, shad...
4559    [archive-name, x-faq/part3, last-modified, 199...
4560    [nice, answer, question, difference, fed, mand...
4561    [hi, got, gateway, 4dx-33v, trying, configure,...
4562    [h, armenian, nagarno-karabagh, simply, defend...
Name: text, Length: 4563, dtype: object

In [47]:
df["text_pp"] = text_pp
df


,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,"[reading, u.s.c, something, make, sence, wonde..."
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,"[bad, question, ref, list, algorithm, either, ..."
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,"[actuallay, hand, support, idea, one, newsgrou..."
3,"The following problem is really bugging me,\na...",2,comp.windows.x,"[following, problem, really, bugging, would, a..."
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,"[latest, upi, foreign, ministry, spokesman, fe..."
...,...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics,"[hi, everyone, looking, software, called, shad..."
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x,"[archive-name, x-faq/part3, last-modified, 199..."
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns,"[nice, answer, question, difference, fed, mand..."
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x,"[hi, got, gateway, 4dx-33v, trying, configure,..."


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points:2}

**Your tasks:**

Outline the preprocessing steps you carried out in the previous exercise and provide a brief justification for these steps.

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

1. Tokenize each row into words to allow word analysis
2. Remove stopwords, punctuation, numbers, and an additional "n't" stopword to ignore tokens that don't contribute much information
3. Lemmatize words for better generalization so that different forms of the same base word are treated the same.  I initially also stemmed the words, but removed that step to make words more readable and to have more related words in each topic.
4. Added more words to ignore after seeing unrelated topic words in 2.4.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points:4}

**Your tasks:**
1. Create a topic model on the preprocessed data using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html). Pick a reasonable number for `n_components`, i.e., number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

In [48]:
joined = copy.deepcopy(df["text_pp"])
for i in range(rows):
    joined[i] = " ".join(df["text_pp"][i])
pp_X = vec.fit_transform(joined)
pp_X

<4563x42631 sparse matrix of type '<class 'numpy.int64'>'
	with 295943 stored elements in Compressed Sparse Row format>

In [49]:
from sklearn.decomposition import LatentDirichletAllocation

n_components = 4
lda = LatentDirichletAllocation(n_components=n_components, max_iter=10, random_state=0)
lda.fit(pp_X)
topics = lda.transform(pp_X)

In [50]:
topics

array([[7.34698518e-01, 2.55908961e-01, 4.69336615e-03, 4.69915467e-03],
       [8.78703175e-02, 3.72164851e-03, 9.04705072e-01, 3.70296176e-03],
       [6.64174594e-01, 4.56784448e-03, 3.26754699e-01, 4.50286284e-03],
       ...,
       [9.83830312e-01, 5.72938139e-03, 5.19320354e-03, 5.24710288e-03],
       [5.86689931e-03, 5.86234249e-03, 9.82271735e-01, 5.99902366e-03],
       [8.22690933e-04, 9.97573607e-01, 7.96061480e-04, 8.07641022e-04]])

In [51]:
df["target_name"].unique

<bound method Series.unique of 0          talk.politics.guns
1               comp.graphics
2               comp.graphics
3              comp.windows.x
4       talk.politics.mideast
                ...          
4558            comp.graphics
4559           comp.windows.x
4560       talk.politics.guns
4561           comp.windows.x
4562    talk.politics.mideast
Name: target_name, Length: 4563, dtype: object>

I chose 4 components because I wanted to merge the 2 comp categories, 2 rec.sport categories, 2 talk.politics categories, and the atheism and christian categories (religion).  I experimented with more and less, but with less topics the topics had unrelated groups of words, and with more topics there were similar columns.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points:5}

**Your tasks:**
1. Show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [52]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())

import mglearn

mglearn.tools.print_topics(
    topics=range(n_components),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
god           armenian      file          game          
people        people        window        team          
gun           turkish       image         year          
time          time          edu           player        
make          government    program       play          
believe       right         server        entry         
christian     year          available     season        
right         jew           widget        win           
jesus         turkey        version       hockey        
question      went          application   good          




- topic 0: religion and politics
- topic 1: middle east
- topic 2: computers
- topic 3: sports

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points:5}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [53]:
first_five_docs = df.head(5)
first_five_docs

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,"[reading, u.s.c, something, make, sence, wonde..."
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,"[bad, question, ref, list, algorithm, either, ..."
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,"[actuallay, hand, support, idea, one, newsgrou..."
3,"The following problem is really bugging me,\na...",2,comp.windows.x,"[following, problem, really, bugging, would, a..."
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,"[latest, upi, foreign, ministry, spokesman, fe..."


In [54]:
topic_labels = ["religion and politics", "middle east", "computers", "sports"]
first_five_topics = topics[:5]
pd.DataFrame(first_five_topics, index=first_five_docs['text'], columns=topic_labels).assign(sum=first_five_topics.sum(axis=1))


,religion and politics,middle east,computers,sports,sum
text,,,,,
"You know, I was reading 18 U.S.C. 922 and something just did not make \nsence and I was wondering if someone could help me out.\n\nSay U.S.C. 922 :\n\n(1) Except as provided in paragraph (2), it shall be unlawful for\nany person to transfer or possess a machinegun.\n\n Well I got to looking in my law dictionary and I found that a ""person"" \nmight also be an artificial entity that is created by government \nand has no rights under the federal constitution. So, what I \ndon't understand is how a statute like 922 can be enforced on \nan individual. So someone tell me how my government can tell\nme what I can or cannot possess. Just passing a law \ndoes not make it LAW. Everyone knows that laws are constitional\nuntil it goes to court. So, has it ever gone to court, not\njust your run of the mill ""Ok I had it I am guilty, put me in jail""\n\nHas anyone ever claimed that they had a right to possess and was told\nby the Supreme Court that they didn't have that right?\n\n\n",0.734699,0.255909,0.004693,0.004699,1.0
"\n\n\nIt's not a bad question: I don't have any refs that list this algorithm\neither. But thinking about it a bit, it shouldn't be too hard.\n\n1) Take three of the points and find the plane they define as well as\nthe circle that they lie on (you say you have this algorithm already)\n\n2) Find the center of this circle. The line passing through this center\nperpendicular to the plane of the three points passes through the center of\nthe sphere.\n\n3) Repeat with the unused point and two of the original points. This\ngives you two different lines that both pass through the sphere's\norigin. Their interection is the center of the sphere.\n\n4) the radius is easy to compute, it's just the distance from the center to\nany of the original points.\n\nI'll leave the math to you, but this is a workable algorithm. :-)\n\n\nAn alternate method would be to take pairs of points: the plane formed\nby the perpendicular bisector of each line segment pair also contains the\ncenter of the sphere. Three pairs will form three planes, intersecting\nat a point. This might be easier to implement.",0.087870,0.003722,0.904705,0.003703,1.0
"\nActuallay I don't, but on the other hand I don't support the idea of having\none newsgroup for every aspect of graphics programming as proposed by Brian,\nin his reply to my original posting.\nI would suggest a looser structure more like a comp.graphics.programmer,\ncomp.graphics.hw_and_sw\nThe reason for making as few groups as possible is for the same reason you\nsay we shouldn't spilt up, not to get to few postings every day.\nI takes to much time to browse through all postings just to find two or \nthree I'm interested in.\n\nI understand and agree when you say you want all aspects of graphics in one\nmeeting. I agree to some extension. I see news as a forum to exchange ideas,\nhelp others or to be helped. I think this is difficult to achive if there\nare so many different things in one meeting.\n\nGood evening netters|-)",0.664175,0.004568,0.326755,0.004503,1.0
"The following problem is really bugging me,\nand I would appreciate any help.\n\nI create two windows:\n\nw1 (child to root) with event_mask = ButtonPressMask|KeyPressMask;\nw2 (child to w1) with do_not_propagate_mask = ButtonPressMask|KeyPressMask;\n\n\nKeypress events in w2 are discarded, but ButtonPress events fall through\nto w1, with subwindow set to w2.\n\nFYI, I'm using xnews/olvwm.\n\nAm I doing something fundamentally wrong here?",0.066534,0.006913,0.919840,0.006714,1.0
\n\n This is the latest from UPI \n\n Foreign Ministry spokesman Ferhat Ataman told journalists Turkey was\n closing its air space to all flights to and from Armenia and would\n prevent humanitarian aid from reaching the republic overland across\n Turkish territory.\n\n \n Historically even the most uncivilized of peoples have exhibited \n signs of compassion by allowing humanitarian aid to reach civi

Rows 0, 1, 3, 4 are very correct in their topic assignments, as my topic label matches the target name.  Row 2 seems to be moderately incorrect, as it still guesses 33% the correct label of computers, but guesses 66% in the religion and politics topic despite not having that topic's top words (e.g. god, people, gun).

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points:6}

1. In lecture 19, we talked about multi-class classification. Comment on how each model in the list below might be handling multiclass classification. Check `scikit-learn` documentation for each of these models when you answer this question.  
    - Decision Tree
    - KNN
    - Random Forest    
    - Logistic Regression
    - SVM RBF
2. What is transfer learning in natural language processing or computer vision? Briefly explain.     
3. In Lecture 19 we briefly discussed how neural networks are sort of like `Pipeline`s, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

1.
- Decision Tree: Train tree classifiers to differentiate classes with OVR to make a prediction on the highest scoring class.
- KNN: Train models to predict between classes with OVO, where the model votes on the nearest neighbours' class, and take the majority votes.
- Random Forest: Similar to Decision Tree, we would train trees to pick out classes using OVR, just with an ensemble of trees and randomization rather than a single one.
- Logistic Regression: Train binary models to predict classes against the rest using OVR.
- SVM RBF: Train binary models for each pair of classes in OVO and vote on a prediction.

2. Transfer learning in NLP and computer vision is the process of taking an existing pre-trained model and using it as a starting point for our own use.  We are taking advantage of a model in which the ones who built it have trained it on an often large and rich dataset (text corpus in NLP and image corpus in computer vision), which saves us the time, effort, and resources required to do so ourselves.  It is called transfer learning because we are taking the model which has done its learning and transferring its knowledge into our project.  We can either use it straight out of the box, extract features with it, or tune it however we need.

3. The common sequential structure of pipelines and neural networks is useful in transfer learning as it allows us to add steps to an existing pre-trained network or model, as if we were extending a pipeline.  This means that we can take the existing "pipeline" and extract features or specialize it further to fit our task, just as the later steps of the pipeline or layers in the network are specializations of earlier steps or layers, and earlier steps or layers are generalizations of the later ones.

<!-- END QUESTION -->

<br><br><br><br>

**PLEASE READ BEFORE YOU SUBMIT:** 

When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from "1" will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 
4. Make sure that the plots and output are rendered properly in your submitted file. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb so that the TAs can view your submission on Gradescope. 

![](img/eva-well-done.png)